In [106]:
import pandas as pd
from spacy.lang.id import Indonesian
from spacy.lang.id.stop_words import STOP_WORDS

In [107]:
nlp = Indonesian()  # use directly

In [108]:
df = pd.read_csv("clean_jokowi.csv", encoding = "ISO-8859-1", names = ['tweet', 'sentiment'], na_values= ' ')
df.head()


,tweet,sentiment
NaN,sentiment,label
0.0,PARAHNYA DIKASI TEPUK TANGAN LAGI BOS,0.0
1.0,Hai Berjumpa lagi dengan KontraS yang akan kem...,1.0
2.0,Bohong lagi bohong lagi Mikrofon dimatikan dul...,0.0
3.0,Prawobo sentil Jokowi Ugal2an Lalu Bung Adian ...,0.0


In [109]:
stopwords = list(STOP_WORDS)
stopwords

['kiranya',
 'sekarang',
 'kapankah',
 'padahal',
 'berbagai',
 'lah',
 'mengakhiri',
 'kala',
 'dimaksud',
 'tiap',
 'ditanyakan',
 'sejenak',
 'maupun',
 'baik',
 'mula',
 'hari',
 'jawaban',
 'dimulai',
 'diucapkan',
 'bermaksud',
 'berakhirlah',
 'kan',
 'sejumlah',
 'apaan',
 'rata',
 'semua',
 'semuanya',
 'justru',
 'jadilah',
 'ingat',
 'kedua',
 'sebaiknya',
 'seenaknya',
 'cukuplah',
 'ditanya',
 'betulkah',
 'hal',
 'seluruh',
 'yaitu',
 'berkali-kali',
 'seingat',
 'ungkapnya',
 'kasus',
 'melihatnya',
 'sebab',
 'ternyata',
 'belakangan',
 'sekali-kali',
 'waktunya',
 'baru',
 'kenapa',
 'tadinya',
 'bolehkah',
 'waktu',
 'menunjuk',
 'lewat',
 'haruslah',
 'sedikit',
 'umum',
 'seringnya',
 'tampak',
 'menantikan',
 'ucap',
 'berkata',
 'terjadi',
 'inginkan',
 'inilah',
 'mengungkapkan',
 'dituturkannya',
 'lebih',
 'ditunjuk',
 'macam',
 'semaunya',
 'tidakkah',
 'jadinya',
 'penting',
 'dilalui',
 'katakanlah',
 'setelah',
 'dimaksudnya',
 'memperlihatkan',
 'sepihak',

###### Getting Lemma and Stop words

In [110]:
docx = nlp("Budi membaca sebuah Buku")

In [111]:
# Lemmatizing of tokens
for word in docx:
    print(word.text,"Lemma =>",word.lemma_)

Budi Lemma => Budi
membaca Lemma => baca
sebuah Lemma => sebuah
Buku Lemma => Buku


In [112]:
# Filtering out Stopwords and Punctuations
for word in docx:
    if word.is_stop == False and not word.is_punct:
        if word.is_stop != True and not word.is_punct:
            print(word)

Budi
membaca
Buku


In [113]:
# Stop words and Punctuation In List Comprehension
[ word for word in docx if word.is_stop == False and not word.is_punct ]

[Budi, membaca, Buku]

In [114]:
# Use the punctuations of string module
import string
punctuations = string.punctuation

In [115]:
# Creating a Spacy Parser
parser = Indonesian()

In [116]:
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

#### Machine Learning With SKlearn

In [117]:
# ML Packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection

In [118]:
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}
class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [119]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()
nbClassifier = GaussianNB()
mNbClassifier = MultinomialNB()
svmClassifier = svm.SVC()

votingClassifier = VotingClassifier(estimators=[('LinearSVC', classifier),
                                                ('MultinomialNaiveBayes', mNbClassifier), 
                                                ('SVM', svmClassifier)])

In [120]:
# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [121]:
# Splitting Data Set
from sklearn.model_selection import train_test_split
# Features and Labels
X = df['tweet']
ylabels = df['sentiment']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

In [123]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('to_dense', DenseTransformer()),
                 ('classifier', mNbClassifier)])

In [124]:
# Fit our data
pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('cleaner', <__main__.predictors object at 0x1a238afac8>), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngra...bject at 0x1a238af9e8>), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [125]:
# Predicting with a test dataset
sample_prediction = pipe.predict(X_test)

In [126]:
# Prediction Results
# 1 = Positive review
# 0 = Negative review
for (sample,pred) in zip(X_test,sample_prediction):
    print(sample,"Prediction=>",pred)

Untuk kesekian kali nya jubir jubir jokowi di buat tak berkutik dan menahan rasa malu dan Aktor Protagonis yg melakukan Prediction=> 0.0
2 9jokowilanjut Prediction=> 1.0
TetapIndonesia JokowiLagi Prediction=> 1.0
cek gt Gara gara Iklan Videotron Jokowi Amin Dikejar Syahroni Sidang dugaan pelangga Prediction=> 0.0
Ssstt Ssstt HATTRICK Prediction=> 0.0
Kpop bagus Via valen bagus Nela karisma bagus Jokowi bagus Prabowo bagus Toko bagus Yang jelek cuman tulisan tangan saya Prediction=> 1.0
Yok bisa yok Siaran Pers Peluncuran Laporan Evaluasi 4 Tahun Pemerintahan JK di sektor HAM oleh KontraS Jumat 9 Ok Prediction=> 1.0
Ini logika yg merusak pak karena nyalahin padahal maksudnya nenbak gubernur DKI sebelumnya kan Waspada Prediction=> 0.0
JokowiLagi Prediction=> 1.0
JokowiAmin jokowimarufamin Prediction=> 0.0
Saya menyampaikan selamat kepada sahabat saya Bapak Prabowo Subianto yang merayakan ulang tahun yang ke 67 pada hari ini Ter Prediction=> 0.0
Di Sumbawa saya saksikan bantuan pemerintah

In [127]:
# Accuracy
print("Accuracy: ",pipe.score(X_test,y_test))
print("Accuracy: ",pipe.score(X_test,sample_prediction))

Accuracy:  0.7441860465116279
Accuracy:  1.0


In [128]:
# Accuracy
print("Accuracy: ",pipe.score(X_train,y_train))

Accuracy:  0.9883720930232558


In [129]:
# Another random review
# pipe.predict(["jokowi Siapa pun Cawapres intinya Jokowi harus 2 periode"])
pipe.predict(["Memilih Jokowi sebagai pemimpin dengan alasan tak ingin Prabowo berkuasa adalah hal paling menyedihkan yg pernah gw dengar"])

array(['0.0'], dtype='<U5')